In [1]:
using LinearAlgebra
using QuantumOptics
using Plots
plotly()
using Printf 
import QuantumOptics.steadystate as steady

In [135]:
grJ=1
exJ=0
ΩL=1
ΩR=1
γ=0.05
Γ=1
Δ=0.5
br=0.5
myJ1system=NLevelBasis(4)

NLevel(N=4)

In [148]:
Bmin=-5.0
Bmax=5.0
Bstep=0.1

function calc_obs(t,ρ)
    abs=3*Γ*imag(expect(σ₁-σ₂,ρ))
    phase=3*Γ*real(expect(σ₁-σ₂,ρ))
    rot=3*Γ*real(expect(σ₁+σ₂,ρ))
    ell=3*Γ*imag(expect(σ₁+σ₂,ρ))
end

σ₁₄=transition(myJ1system,1,4)
σ₃₄=transition(myJ1system,3,4)
γ₄₁=transition(myJ1system,4,1)
γ₄₂=transition(myJ1system,4,2)
γ₄₃=transition(myJ1system,4,3)
proj1=transition(myJ1system,1,1)
proj2=transition(myJ1system,2,2)
proj3=transition(myJ1system,3,3)
proj4=transition(myJ1system,4,4)
id=identityoperator(proj1)
J=[sqrt(γ)*proj1,
   sqrt(γ)*proj2,
   sqrt(γ)*proj3,
   sqrt(γ+Γ)*proj4,
   sqrt(br*Γ/3)*γ₄₁,
   sqrt(br*Γ/3)*γ₄₂,
   sqrt(br*Γ/3)*γ₄₃,
   sqrt(γ/3)*id]
rates=[-γ+γ/3,
       -γ+γ/3,
       -γ+γ/3,
       -(γ+Γ),
       (br*Γ/3),
       (br*Γ/3),
       (br*Γ/3)]
Bvalues=[]
obs1=[]
obs2=[]
obs3=[]
obs4=[]
for B=Bmin:Bstep:Bmax
    H=(ΩR)*(σ₁₄+dagger(σ₁₄)) + (ΩR)*(σ₃₄+dagger(σ₃₄)) + ΩL*B*proj1 - ΩL*B*proj3 - Δ*proj4
    ϕ₀=dm((nlevelstate(myJ1system,1)+nlevelstate(myJ1system,3))/2)
    t_out, ρ_master = steady.master(H,J;rho0=ϕ₀)
    ell=3*(Γ/ΩR)*imag(expect(σ₁₄-σ₃₄,ρ_master[2]))
    rot=3*(Γ/ΩR)*real(expect(σ₁₄-σ₃₄,ρ_master[2]))
    abs=3*(Γ/ΩR)*real(expect(σ₁₄+σ₃₄,ρ_master[2]))
    Δϕ=3*(Γ/ΩR)*imag(expect(σ₁₄+σ₃₄,ρ_master[2]))
    push!(Bvalues,B)
    push!(obs1,abs)
    push!(obs2,Δϕ)
    push!(obs3,rot)
    push!(obs4,ell)
end

In [149]:
Plots.plot(Bvalues,obs1)

<!DOCTYPE html>
 
 
 Plots.jl

In [147]:
identityoperator(proj1)

Operator(dim=4x4)
  basis: NLevel(N=4)
 1.0+0.0im      ⋅          ⋅          ⋅    
     ⋅      1.0+0.0im      ⋅          ⋅    
     ⋅          ⋅      1.0+0.0im      ⋅    
     ⋅          ⋅          ⋅      1.0+0.0im

In [7]:
imag(x)

4